<a href="https://colab.research.google.com/github/mssamoilenko/AnlDataPract/blob/main/PraktAnl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практична робота №3
## Тема: класифікація за допомогою логістичної
## регресії та наївного Баєса.
## Мета: закріплення навичок роботи із категорійними даними, їх по
## передньої обробки та класифікація даних.
## Завдання
## 1.    Використати дані відповідно до власного варіанта (в мене це варіант №2). Вивести таблицю за допомогою Pandas.
## 2.   Визначити числові, категорійні впорядковані, категорійні невпорядковані
## змінні. Для кожної категорійної змінної написати код, який виводить всі
## унікальні значення, котрі вона набуває в наборі даних.
## 3.   Закодувати категорійні впорядковані числами 0, 1, 2… (або іншими
## числами аналогічним чином), до невпорядкованих застосувати опорне
## кодування.
## 4.   Застосувати методи логістичної регресії та наївного Баєса до оброблених
## даних для передбачення змінної «Статус проєкта». На крос-валідації обрати
## кращу модель.
## Варіант роботи № 2
## Хід роботи

## 1.   Виводжу таблицю з даними відповідно моєму варіанту.







In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
df = pd.read_csv("2.csv")
df

,Потужність (л/г),Тара,Якість,Структура,Відстань від джерела (км),Матеріал,Вартість (грн),Статус проєкта
0,38.33,Особливе замовлення,Низька,Ромбовидна,2479.88,Скло,954.65,Заблоковано
1,76.99,Особливе замовлення,Дуже низька,Фрактальна,1825.69,Залізо,1216.99,Узгоджено
2,48.72,Каністра,Дуже низька,Решітка,2034.32,Залізо,861.93,Узгоджено
3,63.23,Своя,Низька,Ромбовидна,2518.65,Поліпропілен,1055.64,Заблоковано
4,24.63,Бочка,Дуже низька,Фрактальна,2661.19,Скло,1063.26,Заблоковано
...,...,...,...,...,...,...,...,...
4995,37.36,Бочка,Висока,Решітка,2462.35,Льон,911.51,Заблоковано
4996,41.68,Бочка,Висока,Фрактальна,2774.46,Залізо,1136.20,Заблоковано
4997,51.92,Каністра,Середня,Ромбовидна,2482.63,Льон,1056.54,Узгоджено
4998,49.98,Каністра,Середня,Фрактальна,1768.00,Поліпропілен,1134.95,Узгоджено


## 2. Визначаю числові, категорійні впорядковані, категорійні невпорядковані змінні. Для кожної категорійної змінної пишу код, який виводить всі унікальні значення, котрі вона набуває в наборі даних.

In [83]:
# 1. Числові змінні
numerical_vars = ['Потужність (л/г)', 'Відстань від джерела (км)', 'Вартість (грн)']

In [84]:
# 2. Категорійні впорядковані змінні
ordinal_vars = ['Якість']

In [85]:
# 3. Категорійні невпорядковані змінні
categorical_vars = ['Тара', 'Структура', 'Матеріал', 'Статус проєкта']

In [86]:
df['Якість'].unique()

array(['Низька', 'Дуже низька', 'Середня', 'Висока'], dtype=object)

In [87]:
df['Тара'].unique()

array(['Особливе замовлення', 'Каністра', 'Своя', 'Бочка'], dtype=object)

In [88]:
df['Структура'].unique()

array(['Ромбовидна', 'Фрактальна', 'Решітка'], dtype=object)

In [89]:
df['Матеріал'].unique()

array(['Скло', 'Залізо', 'Поліпропілен', 'Льон'], dtype=object)

In [90]:
df['Статус проєкта'].unique()

array(['Заблоковано', 'Узгоджено'], dtype=object)

## 3. Я здійснила кодування категорійних ознак: впорядковані змінні закодовано числовими значеннями (0, 1, 2, 3 відповідно до рівня ознаки), а до невпорядкованих застосовано опорне (one-hot) кодування.

In [91]:
# Копіюємо набір
df_encoded = df.copy()

In [92]:
quality_order = {'Дуже низька': 0, 'Низька': 1, 'Середня': 2, 'Висока': 3}
df_encoded['Якість'] = df_encoded['Якість'].map(quality_order)

In [93]:
# One-hot encoding для решти змінних
df_encoded = pd.get_dummies(
    df_encoded,
    columns=['Тара', 'Структура', 'Матеріал', 'Статус проєкта'],
    drop_first=True,
    dtype=int
)

In [94]:
df_encoded.head()

,Потужність (л/г),Якість,Відстань від джерела (км),Вартість (грн),Тара_Каністра,Тара_Особливе замовлення,Тара_Своя,Структура_Ромбовидна,Структура_Фрактальна,Матеріал_Льон,Матеріал_Поліпропілен,Матеріал_Скло,Статус проєкта_Узгоджено
0,38.33,1,2479.88,954.65,0,1,0,1,0,0,0,1,0
1,76.99,0,1825.69,1216.99,0,1,0,0,1,0,0,0,1
2,48.72,0,2034.32,861.93,1,0,0,0,0,0,0,0,1
3,63.23,1,2518.65,1055.64,0,0,1,1,0,0,1,0,0
4,24.63,0,2661.19,1063.26,0,0,0,0,1,0,0,1,0


## 4.  Застосовую методи логістичної регресії та наївного Баєса до оброблених даних для передбачення змінної «Статус проєкта». На крос-валідації обераю кращу модель.

In [95]:
# 2. Визначаю ознаки (X) та цільову змінну (y)
X = df_encoded.drop('Статус проєкта_Узгоджено', axis=1)
y = df_encoded['Статус проєкта_Узгоджено']

In [96]:
# 3. Розділяю дані на тестувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [97]:
# 4. Масштабую числові ознаки (для логістичної регресії це важливо)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # fit_transform тільки на тренувальних!
X_test_scaled = scaler.transform(X_test)        # transform на тестових

In [98]:
# Створюю моделі
log_reg = LogisticRegression(max_iter=1000, random_state=42)
nb = GaussianNB()

In [99]:
# 5. Крос-валідація на ТРЕНУВАЛЬНИХ даних
cv_log = cross_val_score(log_reg, X_train_scaled, y_train, cv=5, scoring='accuracy')
cv_nb = cross_val_score(nb, X_train_scaled, y_train, cv=5, scoring='accuracy')

In [100]:
# 6. Вивід результатів крос-валідації
print("Середня точність логістичної регресії (CV):", cv_log.mean())
print("Середня точність наївного Байєса (CV):", cv_nb.mean())

Середня точність логістичної регресії (CV): 0.99375
Середня точність наївного Байєса (CV): 0.94525


In [101]:
# 7. Фінальна оцінка на тестовій вибірці
log_reg.fit(X_train_scaled, y_train)
nb.fit(X_train_scaled, y_train)

y_pred_log = log_reg.predict(X_test_scaled)
y_pred_nb = nb.predict(X_test_scaled)

test_accuracy_log = accuracy_score(y_test, y_pred_log)
test_accuracy_nb = accuracy_score(y_test, y_pred_nb)

print("\nТочність на тестовій вибірці:")
print("Логістична регресія:", test_accuracy_log)
print("Наївний Байєс:", test_accuracy_nb)


Точність на тестовій вибірці:
Логістична регресія: 0.994
Наївний Байєс: 0.954


In [102]:
# 8. Визначення кращої моделі на основі тестової вибірки
if test_accuracy_log > test_accuracy_nb:
    print("\nКраща модель: Логістична регресія")
else:
    print("\nКраща модель: Наївний Байєс")


Краща модель: Логістична регресія


# Висновки:
## У ході виконання практичної роботи було проведено класифікацію даних з використанням логістичної регресії та наївного Баєса. Основні етапи роботи включали:
## 1.   Завантаження та аналіз даних: Було завантажено набір даних з 5000 записів та 8 ознак, включаючи як числові, так і категорійні змінні.
## 2.   Ідентифікація типів змінних:

## *   Числові змінні: "Потужність (л/г)", "Відстань від джерела (км)", "Вартість (грн)"
## *   Категорійні впорядковані: "Якість" (з унікальними значеннями: 'Низька', 'Дуже низька', 'Середня', 'Висока')
## *   Категорійні невпорядковані: "Тара", "Структура", "Матеріал", "Статус проєкта"
## 3.   Попередня обробка даних: Впорядковані категорійні змінні закодовано числовими значеннями (0, 1, 2, 3); Невпорядковані категорійні змінні оброблено за допомогою one-hot кодування.
## 4.   Побудова та оцінка моделей.

## Результат: Логістична регресія продемонструвала вищу ефективність для класифікації статусу проекту в даному наборі даних.










# Відповіді на контрольні питання
1. Фіктивні змінні - це штучно створені змінні, що використовуються для представлення категорійних даних у числовому форматі. Вони дозволяють включати категорійні ознаки в математичні моделі.

2. Різниця між опорним кодуванням та кодуванням з одним активним станом:

* Кодування з одним активним станом (one-hot) створює окрему бінарну змінну для кожної категорії

* Опорне кодування (dummy coding) використовує k-1 змінну для k категорій, де одна категорія є базовою (опорною)

3. Параметр drop_first=True важливий для уникнення мультиколінеарності - ситуації, коли одна зміння може бути точно виражена через лінійну комбінацію інших змінних.

4. Типи категоріальних змінних:

* Впорядковані (ordinal) - мають природний порядок (наприклад, освіта: середня, вища)

* Невпорядковані (nominal) - не мають природного порядку (наприклад, колір, марка автомобіля)

5. Кодування впорядкованих категоріальних змінних здійснюється шляхом присвоєння числових значень з урахуванням порядку (наприклад, 0, 1, 2, 3 для рівнів якості).

6. Логістична регресія вважається методом класифікації, оскільки вона прогнозує ймовірність приналежності до певного класу, а не безпосередньо числове значення.

7. Ключове припущення наївного Баєса - незалежність ознак при фіксованому класі (припущення про умовну незалежність).

8. Ліниві учні повільні під час прогнозування, оскільки вони не будують модель під час навчання, а зберігають всі навчальні дані та виконують обчислення для кожного нового прикладу.

9. Головна мета крос-валідації - оцінка узагальнюючої здатності моделі на небачених даних та запобігання перенавчанню.

10. Проблеми при відсутності кодування категоріальних змінних:

* Більшість алгоритмів машинного навчання не можуть обробляти текстовий формат безпосередньо

* Модель може інтерпретувати категорії як впорядковані, коли вони такими не є

* Можливі помилки при роботі з алгоритмами, що вимагають числових вхідних даних